### importing requried libraries

In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import cv2, numpy as np
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LambdaCallback

Using TensorFlow backend.


### Defining the Deep Learning Model

In [2]:
l2_reg = 0.001

def mymodel_2(weights_path=None,kernel_regularizer=l2(l2_reg)):
    model = Sequential()

    model.add(ZeroPadding2D((1,1),input_shape=(124,124,1)))
    
    model.add(Convolution2D(6, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(16, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(32, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    '''model.add(Convolution2D(64, 7, 7))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
'''
    '''model.add(Convolution2D(128, 3, 3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))'''
    
    model.add(Flatten())
    #model.add(Dense(1024))
    model.add(Dense(128))
    model.add(Activation('relu'))
    #model.add(Dropout(0.3))'''
    
    #model.add(Dense(856))
    model.add(Dense(64))
    model.add(Activation('relu'))
    #model.add(Dropout(0.3))
    
    #model.add(Dense(556))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model

### Creating Image Data Generator

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

#zca_whitening=True
#featurewise_center=True, featurewise_std_normalization=True

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zca_whitening=True,
    featurewise_center=True, 
    featurewise_std_normalization=True,    
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip= True,                               
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    '/home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/images for training/',
    target_size=(124, 124),
    batch_size=32,
    color_mode = "grayscale",
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    '/home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/images for training/', # same directory as training data
    target_size=(124, 124),
    batch_size=32,
    color_mode = "grayscale",
    class_mode='categorical',
    subset='validation') # set as validation data



/home/abhi/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:339: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Found 840 images belonging to 2 classes.
Found 210 images belonging to 2 classes.


In [4]:
model = mymodel_2()

Instructions for updating:
Colocations handled automatically by placer.


/home/abhi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (3, 3))`
  
/home/abhi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3))`
  del sys.path[0]
/home/abhi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 126, 126, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 6)       60        
_________________________________________________________________
batch_normalization_1 (Batch (None, 124, 124, 6)       24        
_________________________________________________________________
activation_1 (Activation)    (None, 124, 124, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 16)        880       
_________________________________________________________________
batch_normalization_2 (Batch (None, 60, 60, 16)        64        
__________

In [6]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import time

opt = optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, decay=1e-4/30, amsgrad=False, clipnorm = 1.)

model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [7]:
mc = ModelCheckpoint('/home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/weights of training/NEWweights{epoch:08d}.hdf5',monitor='acc',
                     verbose=1,  mode='max',
                                     save_weights_only=False, period=1)

In [8]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    #callbacks=[tensorB],
    #callbacks=[TensorBoardColabCallback(tbc)],
    callbacks=[mc],
    epochs = 200)
#model.save("mymodel_1_e10_1.h5")

Instructions for updating:
Use tf.cast instead.
Epoch 1/200


/home/abhi/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/abhi/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


26/26 [==============================] - 27s 1s/step - loss: 0.4372 - acc: 0.8065 - val_loss: 0.1081 - val_acc: 0.9792

Epoch 00001: saving model to /home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/weights of training/NEWweights00000001.hdf5
Epoch 2/200
26/26 [==============================] - 23s 871ms/step - loss: 0.1178 - acc: 0.9568 - val_loss: 0.0527 - val_acc: 0.9663

Epoch 00002: saving model to /home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/weights of training/NEWweights00000002.hdf5
Epoch 3/200
26/26 [==============================] - 22s 847ms/step - loss: 0.0625 - acc: 0.9723 - val_loss: 0.0283 - val_acc: 0.9831

Epoch 00003: saving model to /home/abhi/Documents/RedBot technologies/BREKS IMAGES 562 GRADE/weights of training/NEWweights00000003.hdf5
Epoch 4/200
26/26 [==============================] - 22s 833ms/step - loss: 0.0326 - acc: 0.9904 - val_loss: 0.0155 - val_acc: 0.9944

Epoch 00004: saving model to /home/abhi/Documents/RedBot technol

KeyboardInterrupt: 